In [6]:
import psycopg2
import pandas as pd

# Connect to PostgreSQL database
def connect_to_postgres():
    conn = psycopg2.connect(
        host="localhost",       
        database="lombardia_air_quality", 
        user="airdata_user",    
        password="user"
    )
    return conn


In [ ]:


def query_data(query, format='json'):
    """
    Esegue una query su PostgreSQL e restituendo DataFrame o JSON (lista di dict). Parametri:
        query (str): Query SQL da eseguire
        format (str): 'dataframe' o 'json'
    """
    try:
        conn = connect_to_postgres()
        if format == 'dataframe':
            df = pd.read_sql_query(query, conn)
            return df
        
        elif format == 'json':
            cur = conn.cursor()
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            rows = cur.fetchall()
            return [dict(zip(columns, row)) for row in rows]
        
        else:
            raise ValueError("format deve essere 'dataframe' o 'json'")

    except Exception as e:
        print(f"Errore: {e}")
        return None

    finally:
        if conn:
            conn.close()


In [10]:

# --- Esempio di utilizzo ---
# Esegui una query SQL
sql_query = "SELECT * FROM users;"

# Ricevere JSON
json_data = query_data(sql_query, format='json')
print('JSON:')
print(json_data)

# Ricevere un DataFrame
df = query_data(sql_query, format='dataframe')
print('Dataframe:')
print(df.head(), '\n') 
# ignore userWARNING


JSON:
[{'id': 1, 'username': 'admin', 'email': 'admin@admin.com', 'password': 'admin', 'type': 'admin', 'created_at': datetime.datetime(2025, 5, 26, 15, 34, 30, 770897)}, {'id': 2, 'username': 'user name', 'email': 'user@email.com', 'password': 'user', 'type': 'user', 'created_at': datetime.datetime(2025, 5, 26, 15, 34, 30, 770897)}, {'id': 3, 'username': 'test', 'email': 'test@test.com', 'password': 'test', 'type': 'user', 'created_at': datetime.datetime(2025, 5, 26, 15, 34, 30, 770897)}]
Dataframe:
   id   username            email password   type                 created_at
0   1      admin  admin@admin.com    admin  admin 2025-05-26 15:34:30.770897
1   2  user name   user@email.com     user   user 2025-05-26 15:34:30.770897
2   3       test    test@test.com     test   user 2025-05-26 15:34:30.770897 



C:\Users\User\AppData\Local\Temp\ipykernel_14088\3578338141.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:
# Alternative without df and json conversion
def select_query(sql, data=None):
    """Esegue una SELECT e restituisce tutti i risultati."""
    try:
        with connect_to_postgres() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, data) if data else cur.execute(sql)
                return cur.fetchall()
    except Exception as e:
        print(f"Errore SELECT: {e}")
        return []
    

In [ ]:
query="SELECT * FROM users WHERE id = %s OR id = %s;"
select_query(query, (1,2))  # Esempio di utilizzo della funzione select_query

[(1,
  'admin',
  'admin@admin.com',
  'admin',
  'admin',
  datetime.datetime(2025, 5, 26, 15, 34, 30, 770897)),
 (2,
  'user name',
  'user@email.com',
  'user',
  'user',
  datetime.datetime(2025, 5, 26, 15, 34, 30, 770897))]

In [16]:
# Alternative without df and json conversion
def select_query(username, email, password, user_type='user'):
    """Esegue una SELECT e restituisce tutti i risultati."""
    sql1 = "SELECT id, username, email, type FROM users WHERE username = %s;"
    sql2 = """
            INSERT INTO users (username, email, password, type)
            VALUES (%s, %s, %s, %s)
            RETURNING id, username, email, type;
            """
    try:
        with connect_to_postgres() as conn:
            with conn.cursor() as cur:
                cur.execute(sql1, (username,))
                user = cur.fetchone()
                # 1. Se esiste, ritorna tupla es: (1, 'mario', 'mario@example.com', 'admin')
                if user:
                    print("Utente esistente trovato.")
                    return user
                # 2. Se non esiste, inserisce
                cur.execute(sql2,(username, email, password, user_type,))
                new_user = cur.fetchone()
                conn.commit()
                print("Nuovo utente inserito.")
                return new_user
            
    except Exception as e:
        print(f"Errore SELECT: {e}")
        return []
    
# Esempio di utilizzo della funzione select_query
new_user = select_query('mario', 'mario@gmail.com', 'password123')

Utente esistente trovato.
